In [1]:
from transformers import BertTokenizer
from prep import get_train_test_dev
import pandas as pd
import numpy as np

In [10]:
train,dev,test = get_train_test_dev()
all_data = pd.concat([train,dev,test])

In [11]:
all_data.head()

,text,perspective,stance_label_3,split
16,Male infant circumcision is tantamount to chil...,Parents have the right to use their best judgm...,UNDERMINE,train
17,Male infant circumcision is tantamount to chil...,Parents know what best for thier child,UNDERMINE,train
18,Male infant circumcision is tantamount to chil...,Parents have the right to make the decisions f...,UNDERMINE,train
24,punishment should fit the criminal,It will cause less re-offenders.,SUPPORT,train
25,punishment should fit the criminal,Adequate punishment reduces future offenses.,SUPPORT,train


In [70]:
all_data[(all_data['text'].str[-1]=='.')]['text'][430]

'Individuals should be able to legally bypass copyright protection technologies for fair use reasons.'

In [58]:
def make_tokenizer(tknzr):
    
    def process_for_bert(sentence:str):
        encoded = tokenizer.encode_plus(
                                        sentence,                    
                                        add_special_tokens = True,
                                        max_length = 64,           
                                        pad_to_max_length = True,
                                        return_attention_mask = True,  
                                        return_tensors = 'pt',
                                        )

        ids = encoded['input_ids'].detach().numpy().reshape(-1)
        attention_mask = encoded['attention_mask'].detach().numpy().reshape(-1)

        return ids,attention_mask
    
    return process_for_bert

In [59]:
preprocceser = make_tokenizer(tokenizer)
a,b = preprocceser('i dont know how to do this')

In [61]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
preprocceser = make_tokenizer(tokenizer)

def tokenize_data_set(data_set,proccese):
    ids_and_attentions = data_set['text'].apply(lambda x:proccese(x))
    tokenized = pd.DataFrame(data=a,columns=['input_ids','attention_mask'])
    full_data_set = pd.concat([data_set,tokenized],axis=1)
    return full_data_set


In [62]:
new_all = tokenize_data_set(all_data,preprocceser)
new_all.head()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ValueError: Shape of passed values is (64, 1), indices imply (64, 2)

In [36]:
new_all['input_ids'][0]

0.0

In [8]:
a = process_for_bert('i dont know how to do this',tokenizer)
a

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(tensor([[ 101, 1045, 2123, 2102, 2113, 2129, 2000, 2079, 2023,  102,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

In [11]:
clf = BertModel.from_pretrained("bert-base-cased",num_labels=3)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
